In [1]:
import pandas as pd

# Define the filenames and variable names
dataset_names = [
    "X_train", 
    "X_validate",
    "X_test", 
    "y_train", "y_validate", "train_df", "test_df"
]

# Load each dataset from CSV
datasets = {name: pd.read_csv(f"{name}.csv") for name in dataset_names}

# Unpack variables for direct usage
X_train = datasets["X_train"]
X_validate = datasets["X_validate"]
X_test = datasets["X_test"]
y_train = datasets["y_train"]
y_validate = datasets["y_validate"]
train_df = datasets["train_df"]
test_df = datasets["test_df"]

print("All datasets loaded successfully! 🎉")


All datasets loaded successfully! 🎉


In [81]:
train_df

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Group,MRP_per_unit_Weight,Outlet_Age,Item_Category,Outlet_Total_Sales,Outlet_Combined,Item_MRP_Per_Unit_Weight,Outlet_Age_Binned
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,FD,26.861204,10,Food,2.183970e+06,Supermarket Type1_Medium,26.861204,New
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,DR,8.153581,0,Drinks,1.851823e+06,Supermarket Type2_Medium,8.153581,New
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,FD,8.092457,10,Food,2.183970e+06,Supermarket Type1_Medium,8.092457,New
3,FDX07,19.200,Regular,0.022861,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800,FD,9.484115,11,Food,1.883402e+05,Grocery Store_Small,9.484115,Moderate
4,NCD19,8.930,Low Fat,0.006590,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,NC,6.031512,22,Non-Consumable,2.142664e+06,Supermarket Type1_High,6.031512,Old
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834,FD,31.248623,22,Food,2.142664e+06,Supermarket Type1_High,31.248623,Old
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,Small,Tier 2,Supermarket Type1,549.2850,FD,12.906563,7,Food,2.036725e+06,Supermarket Type1_Small,12.906563,New
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136,NC,8.030415,5,Non-Consumable,2.268123e+06,Supermarket Type1_Small,8.030415,New
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976,FD,14.304189,0,Food,1.851823e+06,Supermarket Type2_Medium,14.304189,New


In [4]:
import pandas as pd
import numpy as np
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Select features and target
categorical_cols = ['Item_Fat_Content', 'Item_Category', 'Outlet_Location_Type', 'Outlet_Type', 'Outlet_Size']
numerical_cols = ['Item_MRP', 'Outlet_Total_Sales', 'Item_Visibility', 'Item_Weight', 'Outlet_Age']

# Ensure categorical columns are treated as category dtype
for col in categorical_cols:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

# Select X and y
X = train_df[numerical_cols + categorical_cols]
y = train_df['Item_Outlet_Sales']
X_test = test_df[numerical_cols + categorical_cols]

# Split data
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05),
        'num_leaves': trial.suggest_int('num_leaves', 20, 200),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 10.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 10.0),
        'n_estimators': trial.suggest_int('n_estimators', 300, 1500)
        
    }

    model = lgb.LGBMRegressor(**params)

    eval_result = {}
    model.fit(
        X_train, y_train, 
        eval_set=[(X_validate, y_validate)], 
        eval_metric='rmse',
        categorical_feature=categorical_cols,  # Pass categorical features!
        callbacks=[lgb.early_stopping(50, verbose=False), lgb.record_evaluation(eval_result)]
    )
    
    preds = model.predict(X_validate)
    rmse = mean_squared_error(y_validate, preds, squared=False)
    return rmse

# Run Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)  # Increase trials for better tuning

print("Best Parameters:", study.best_params)


[I 2025-03-17 18:17:51,198] A new study created in memory with name: no-name-4eafbe10-1cbe-49a1-b1d1-180a6fa65825


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001505 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 782
[LightGBM] [Info] Number of data points in the train set: 6818, number of used features: 10
[LightGBM] [Info] Start training from score 2202.365232
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-03-17 18:17:51,781] Trial 0 finished with value: 1019.1008238999981 and parameters: {'learning_rate': 0.01069483256791398, 'num_leaves': 126, 'max_depth': 7, 'min_child_samples': 34, 'colsample_bytree': 0.6682917997588329, 'subsample': 0.5549494367258951, 'reg_lambda': 5.042103297374779, 'reg_alpha': 2.9205707667120144, 'n_estimators': 1376}. Best is trial 0 with value: 1019.1008238999981.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[I 2025-03-17 18:18:19,884] Trial 97 finished with value: 1015.5411553430832 and parameters: {'learning_rate': 0.047917939384843976, 'num_leaves': 99, 'max_depth': 7, 'min_child_samples': 28, 'colsample_bytree': 0.8552423344067465, 'subsample': 0.9241498445984972, 'reg_lambda': 9.766263133456068, 'reg_alpha': 8.376617235204144, 'n_estimators': 1341}. Best is trial 80 with value: 1011.370498722554.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000211 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 782
[LightGBM] [Info] Number of data points in the train set: 6818, number of used features: 10
[LightGBM] [Info] Start training from score 2202.365232
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000249 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 782
[LightGBM] [Info] Number of data points in the train set: 6818, number of used features: 10
[LightGBM] [Info] Start training from score 2202.365232
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

[I 2025-03-17 18:18:20,210] Trial 98 finished with value: 1015.9311633963123 and parameters: {'learning_rate': 0.04547723991149968, 'num_leaves': 84, 'max_depth': 8, 'min_child_samples': 34, 'colsample_bytree': 0.9240845157525267, 'subsample': 0.9695297348524359, 'reg_lambda': 9.283983651186686, 'reg_alpha': 5.500602030166416, 'n_estimators': 1233}. Best is trial 80 with value: 1011.370498722554.
[I 2025-03-17 18:18:20,458] Trial 99 finished with value: 1014.1680721482697 and parameters: {'learning_rate': 0.046935832034996926, 'num_leaves': 134, 'max_depth': 6, 'min_child_samples': 36, 'colsample_bytree': 0.955186085700521, 'subsample': 0.9602562907199849, 'reg_lambda': 8.792453949549815, 'reg_alpha': 7.251814700430523, 'n_estimators': 1432}. Best is trial 80 with value: 1011.370498722554.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000291 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 782
[LightGBM] [Info] Number of data points in the train set: 6818, number of used features: 10
[LightGBM] [Info] Start training from score 2202.365232
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [5]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Best parameters from Optuna
best_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.04780732159947716,
    'num_leaves': 122,
    'max_depth': 6,
    'min_child_samples': 32,
    'subsample': 0.9157606719548699,
    'colsample_bytree': 0.8817927721417727,
    'n_estimators': 1305,
    'reg_lambda': 7.883879159201173,
    'reg_alpha': 5.77458194410136,
}


# Train model
model = LGBMRegressor(**best_params)
model.fit(X_train, y_train, 
          eval_set=[(X_validate, y_validate)], 
          eval_metric='rmse', 
          callbacks=[lgb.early_stopping(50, verbose=False)])

# Predict on training & validation sets
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_validate)

# Evaluate performance
def evaluate(y_true, y_pred, dataset):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n✅ Model Performance on {dataset}:")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R² Score: {r2:.4f}")

# Show results
evaluate(y_train, y_train_pred, "Training Set")
evaluate(y_validate, y_valid_pred, "Validation Set")

# Predict on test set
y_test_preds = model.predict(X_test)

# Create submission file
submission = test_df[['Item_Identifier', 'Outlet_Identifier']].copy()
submission['Item_Outlet_Sales'] = y_test_preds

submission['Item_Outlet_Sales'] = y_test_preds.clip(min=0)

# Save CSV
submission.to_csv('../submission/submission_lgb.csv', index=False)

print("Download your submission file: ../submission/submission_lgb.csv")



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000290 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 782
[LightGBM] [Info] Number of data points in the train set: 6818, number of used features: 10
[LightGBM] [Info] Start training from score 2202.365232
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [6]:
import pandas as pd
import numpy as np
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Select features and target
categorical_cols = ['Item_Fat_Content', 'Item_Category', 'Outlet_Location_Type']
numerical_cols = ['Item_MRP', 'Outlet_Total_Sales', 'Item_Visibility', 'Item_Weight', 'Outlet_Age']

# Ensure categorical columns are treated as category dtype
for col in categorical_cols:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

# Select X and y
X = train_df[numerical_cols + categorical_cols]
y = train_df['Item_Outlet_Sales']
X_test = test_df[numerical_cols + categorical_cols]

# Split data
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05),
        'num_leaves': trial.suggest_int('num_leaves', 20, 200),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 10.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 10.0),
        'n_estimators': trial.suggest_int('n_estimators', 300, 1500)
        
    }

    model = lgb.LGBMRegressor(**params)

    eval_result = {}
    model.fit(
        X_train, y_train, 
        eval_set=[(X_validate, y_validate)], 
        eval_metric='rmse',
        categorical_feature=categorical_cols,  # Pass categorical features!
        callbacks=[lgb.early_stopping(50, verbose=False), lgb.record_evaluation(eval_result)]
    )
    
    preds = model.predict(X_validate)
    rmse = mean_squared_error(y_validate, preds, squared=False)
    return rmse

# Run Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)  # Increase trials for better tuning

print("Best Parameters:", study.best_params)


[I 2025-03-17 18:22:08,483] A new study created in memory with name: no-name-2e1efab5-ef2e-4d01-92d3-692b5685d2fc


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000266 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 773
[LightGBM] [Info] Number of data points in the train set: 6818, number of used features: 8
[LightGBM] [Info] Start training from score 2202.365232
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

[I 2025-03-17 18:22:08,797] Trial 0 finished with value: 1020.2342744635603 and parameters: {'learning_rate': 0.04932724915767727, 'num_leaves': 189, 'max_depth': 3, 'min_child_samples': 70, 'colsample_bytree': 0.5122445869595658, 'subsample': 0.6208506653987521, 'reg_lambda': 9.474753723381452, 'reg_alpha': 8.33180634602795, 'n_estimators': 1297}. Best is trial 0 with value: 1020.2342744635603.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-03-17 18:22:09,186] Trial 1 finished with value: 1017.9251530546893 and parameters: {'learning_rate': 0.012908784127561333, 'num_leaves': 68, 'max_depth': 11, 'min_child_samples': 19, 'colsample_bytree': 0.972190445741395, 'subsample': 0.7176069060278654, 'reg_lambda': 1.213713665073071, 'reg_alpha': 7.466222592464927, 'n_estimators': 560}. Best is trial 1 with value: 1017.9251530546893.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-03-17 18:22:34,391] Trial 98 finished with value: 1017.6562574200352 and parameters: {'learning_rate': 0.049030961345047176, 'num_leaves': 177, 'max_depth': 5, 'min_child_samples': 21, 'colsample_bytree': 0.7938964573063568, 'subsample': 0.8034194175862028, 'reg_lambda': 8.87729481805543, 'reg_alpha': 1.2125947278952491, 'n_estimators': 1011}. Best is trial 58 with value: 1012.744209201659.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-03-17 18:22:34,604] Trial 99 finished with value: 1013.9751690751573 and parameters: {'learning_rate': 0.04495054493404202, 'num_leaves': 192, 'max_depth': 5, 'min_child_samples': 48, 'colsample_bytree': 0.8174913313107846, 'subsample': 0.7744291234513374, 'reg_lambda': 9.096704097694102, 'reg_alpha': 0.6895148714604982, 'n_estimators': 1373}. Best is trial 58 with value: 1012.744209201659.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [7]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Best parameters from Optuna
best_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.046008777313723596,
    'num_leaves': 132,
    'max_depth': 6,
    'min_child_samples': 51,
    'subsample': 0.7135667543386197,
    'colsample_bytree': 0.9133800915544624,
    'n_estimators': 1305,
    'reg_lambda': 8.310251218815315,
    'reg_alpha': 4.35964945673663,
}


# Train model
model = LGBMRegressor(**best_params)
model.fit(X_train, y_train, 
          eval_set=[(X_validate, y_validate)], 
          eval_metric='rmse', 
          callbacks=[lgb.early_stopping(50, verbose=False)])

# Predict on training & validation sets
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_validate)

# Evaluate performance
def evaluate(y_true, y_pred, dataset):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n✅ Model Performance on {dataset}:")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R² Score: {r2:.4f}")

# Show results
evaluate(y_train, y_train_pred, "Training Set")
evaluate(y_validate, y_valid_pred, "Validation Set")

# Predict on test set
y_test_preds = model.predict(X_test)

# Create submission file
submission = test_df[['Item_Identifier', 'Outlet_Identifier']].copy()
submission['Item_Outlet_Sales'] = y_test_preds

submission['Item_Outlet_Sales'] = y_test_preds.clip(min=0)

# Save CSV
submission.to_csv('../submission/submission_lgb.csv', index=False)

print("Download your submission file: ../submission/submission_lgb.csv")



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001262 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 773
[LightGBM] [Info] Number of data points in the train set: 6818, number of used features: 8
[LightGBM] [Info] Start training from score 2202.365232
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

Download your submission file: ../submission/submission_lgb.csv


In [25]:

import pandas as pd
import numpy as np
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Create a new combined categorical feature
train_df['Outlet_Combined'] = train_df['Outlet_Location_Type'].astype(str) + "_" + \
                              train_df['Outlet_Type'].astype(str) 
#+ "_" + train_df['Outlet_Size'].astype(str)

test_df['Outlet_Combined'] = test_df['Outlet_Location_Type'].astype(str) + "_" + \
                             test_df['Outlet_Type'].astype(str) 
#+ "_" + test_df['Outlet_Size'].astype(str)

# Select features and target
categorical_cols = [ 'Outlet_Combined']
numerical_cols = ['Item_MRP', 'Outlet_Total_Sales', 'Item_Visibility']

# Ensure categorical columns are treated as category dtype
for col in categorical_cols:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

# Select X and y
X = train_df[numerical_cols + categorical_cols]
y = train_df['Item_Outlet_Sales']
X_test = test_df[numerical_cols + categorical_cols]

# Split data
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.03),  # Wider range
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),  # More flexibility
        'max_depth': trial.suggest_int('max_depth', 3, 7),  # Slightly deeper trees
        'min_child_samples': trial.suggest_int('min_child_samples', 30, 70),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.85),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'reg_lambda': trial.suggest_float('reg_lambda', 5, 12),  # Stronger L2 regularization
        'reg_alpha': trial.suggest_float('reg_alpha', 5, 12),  # Stronger L1 regularization
        'n_estimators': 2000,  # Increased for better convergence
        'verbosity': -1  # Reduce logs for speed
    }

    model = lgb.LGBMRegressor(**params)

    eval_result = {}
    model.fit(
        X_train, y_train, 
        eval_set=[(X_validate, y_validate)], 
        eval_metric='rmse',
        categorical_feature=categorical_cols,  # Pass categorical features!
        callbacks=[lgb.early_stopping(50, verbose=False), lgb.record_evaluation(eval_result)]
    )
    
    preds = model.predict(X_validate)
    rmse = mean_squared_error(y_validate, preds, squared=False)
    return rmse

# Run Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  # Increase trials for better tuning

print("Best Parameters:", study.best_params)


[I 2025-03-17 18:45:45,523] A new study created in memory with name: no-name-2934e974-e2fe-499c-97fa-f881c42ca75f
[I 2025-03-17 18:45:45,614] Trial 0 finished with value: 1018.0355169142222 and parameters: {'learning_rate': 0.02994395892611274, 'num_leaves': 97, 'max_depth': 3, 'min_child_samples': 30, 'colsample_bytree': 0.6952578876810936, 'subsample': 0.7589213499014307, 'reg_lambda': 7.4773952556202286, 'reg_alpha': 10.8623486471046}. Best is trial 0 with value: 1018.0355169142222.
[I 2025-03-17 18:45:45,772] Trial 1 finished with value: 1014.688306792813 and parameters: {'learning_rate': 0.011033018679916705, 'num_leaves': 20, 'max_depth': 4, 'min_child_samples': 67, 'colsample_bytree': 0.8363138237921335, 'subsample': 0.8801384559849327, 'reg_lambda': 9.14341063469826, 'reg_alpha': 5.061369609849074}. Best is trial 1 with value: 1014.688306792813.
[I 2025-03-17 18:45:45,907] Trial 2 finished with value: 1016.6291676282628 and parameters: {'learning_rate': 0.024792064826346525, 'n

[I 2025-03-17 18:45:49,728] Trial 22 finished with value: 1013.8607972400425 and parameters: {'learning_rate': 0.015380994021272918, 'num_leaves': 68, 'max_depth': 4, 'min_child_samples': 63, 'colsample_bytree': 0.6387647474637363, 'subsample': 0.8214060620150229, 'reg_lambda': 9.71769231674056, 'reg_alpha': 10.131564183562702}. Best is trial 18 with value: 1013.0155755741049.
[I 2025-03-17 18:45:49,930] Trial 23 finished with value: 1015.5526229023314 and parameters: {'learning_rate': 0.015701688755469703, 'num_leaves': 70, 'max_depth': 5, 'min_child_samples': 63, 'colsample_bytree': 0.6388396357876576, 'subsample': 0.8165505942515625, 'reg_lambda': 9.93808892663816, 'reg_alpha': 10.26624304676076}. Best is trial 18 with value: 1013.0155755741049.
[I 2025-03-17 18:45:50,052] Trial 24 finished with value: 1016.68355706173 and parameters: {'learning_rate': 0.023582902509868102, 'num_leaves': 85, 'max_depth': 3, 'min_child_samples': 59, 'colsample_bytree': 0.6495955454112402, 'subsample'

[I 2025-03-17 18:45:54,515] Trial 44 finished with value: 1016.3917557927916 and parameters: {'learning_rate': 0.011146942918703304, 'num_leaves': 67, 'max_depth': 4, 'min_child_samples': 54, 'colsample_bytree': 0.6168684347754688, 'subsample': 0.8835885242582276, 'reg_lambda': 5.605926779841074, 'reg_alpha': 10.40296844785581}. Best is trial 18 with value: 1013.0155755741049.
[I 2025-03-17 18:45:54,694] Trial 45 finished with value: 1014.6447279222969 and parameters: {'learning_rate': 0.014574417613815468, 'num_leaves': 82, 'max_depth': 4, 'min_child_samples': 66, 'colsample_bytree': 0.6744483102332407, 'subsample': 0.8601126662339396, 'reg_lambda': 9.266440740967962, 'reg_alpha': 9.817365234375577}. Best is trial 18 with value: 1013.0155755741049.
[I 2025-03-17 18:45:54,870] Trial 46 finished with value: 1016.4505760784001 and parameters: {'learning_rate': 0.013682467484044726, 'num_leaves': 89, 'max_depth': 3, 'min_child_samples': 61, 'colsample_bytree': 0.7302457468029122, 'subsamp

Best Parameters: {'learning_rate': 0.013679293785671084, 'num_leaves': 76, 'max_depth': 4, 'min_child_samples': 63, 'colsample_bytree': 0.625938766116602, 'subsample': 0.8097337683385393, 'reg_lambda': 9.933505007901022, 'reg_alpha': 8.53193027196974}


In [26]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Best parameters from Optuna
best_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.013679293785671084,  
    'num_leaves': 76,  # Slightly more complex trees
    'max_depth': 4,  
    'min_child_samples': 63,  # Allows a bit more flexibility in splits
    'subsample': 0.8097337683385393,  
    'colsample_bytree': 0.625938766116602,  
    'n_estimators': 3000,  
    'reg_lambda': 9.933505007901022,  # Reduce regularization slightly
    'reg_alpha': 8.53193027196974,  # Reduce regularization slightly
}



# Train model
model = LGBMRegressor(**best_params)
model.fit(X_train, y_train, 
          eval_set=[(X_validate, y_validate)], 
          eval_metric='rmse', 
          callbacks=[lgb.early_stopping(50, verbose=False)])

# Predict on training & validation sets
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_validate)

# Evaluate performance
def evaluate(y_true, y_pred, dataset):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n✅ Model Performance on {dataset}:")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R² Score: {r2:.4f}")

# Show results
evaluate(y_train, y_train_pred, "Training Set")
evaluate(y_validate, y_valid_pred, "Validation Set")

# Predict on test set
y_test_preds = model.predict(X_test)

# Create submission file
submission = test_df[['Item_Identifier', 'Outlet_Identifier']].copy()
submission['Item_Outlet_Sales'] = y_test_preds

submission['Item_Outlet_Sales'] = y_test_preds.clip(min=0)

# Save CSV
submission.to_csv('../submission/submission_lgb.csv', index=False)

print("Download your submission file: /mnt/data/submission_xgboost.csv")
#1151.9072619464 rank #1115



✅ Model Performance on Training Set:
MAE: 744.4006
MSE: 1123366.7073
RMSE: 1059.8900
R² Score: 0.6202

✅ Model Performance on Validation Set:
MAE: 711.3731
MSE: 1026200.5564
RMSE: 1013.0156
R² Score: 0.6224
Download your submission file: /mnt/data/submission_xgboost.csv


In [30]:

import pandas as pd
import numpy as np
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Create a new combined categorical feature
train_df['Outlet_Combined'] = train_df['Outlet_Type'].astype(str) \
                                + "_" + train_df['Outlet_Size'].astype(str)
#train_df['Outlet_Location_Type'].astype(str) \
                              #+ "_" + \
                                

test_df['Outlet_Combined'] =    test_df['Outlet_Type'].astype(str) \
                                + "_" + test_df['Outlet_Size'].astype(str)
#test_df['Outlet_Location_Type'].astype(str) \
                              #+ "_" + \

# Select features and target
categorical_cols = [ 'Outlet_Combined']
numerical_cols = ['Item_MRP', 'Outlet_Total_Sales', 'Item_Visibility']

# Ensure categorical columns are treated as category dtype
for col in categorical_cols:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

# Select X and y
X = train_df[numerical_cols + categorical_cols]
y = train_df['Item_Outlet_Sales']
X_test = test_df[numerical_cols + categorical_cols]

# Split data
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.03),  # Wider range
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),  # More flexibility
        'max_depth': trial.suggest_int('max_depth', 3, 7),  # Slightly deeper trees
        'min_child_samples': trial.suggest_int('min_child_samples', 30, 70),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.85),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'reg_lambda': trial.suggest_float('reg_lambda', 5, 12),  # Stronger L2 regularization
        'reg_alpha': trial.suggest_float('reg_alpha', 5, 12),  # Stronger L1 regularization
        'n_estimators': 2000,  # Increased for better convergence
        'verbosity': -1  # Reduce logs for speed
    }

    model = lgb.LGBMRegressor(**params)

    eval_result = {}
    model.fit(
        X_train, y_train, 
        eval_set=[(X_validate, y_validate)], 
        eval_metric='rmse',
        categorical_feature=categorical_cols,  # Pass categorical features!
        callbacks=[lgb.early_stopping(100, verbose=False), lgb.record_evaluation(eval_result)]
    )
    
    preds = model.predict(X_validate)
    rmse = mean_squared_error(y_validate, preds, squared=False)
    return rmse

# Run Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  # Increase trials for better tuning

print("Best Parameters:", study.best_params)


[I 2025-03-17 19:29:24,484] A new study created in memory with name: no-name-aa5b69d7-1cd5-4819-af8a-7b347802bc3a
[I 2025-03-17 19:29:24,625] Trial 0 finished with value: 1017.5585524555383 and parameters: {'learning_rate': 0.02824773334976492, 'num_leaves': 60, 'max_depth': 5, 'min_child_samples': 38, 'colsample_bytree': 0.6789423026558109, 'subsample': 0.7239224319201033, 'reg_lambda': 5.864815071444314, 'reg_alpha': 7.6088163755029}. Best is trial 0 with value: 1017.5585524555383.
[I 2025-03-17 19:29:25,033] Trial 1 finished with value: 1015.6236199327619 and parameters: {'learning_rate': 0.007715645956689507, 'num_leaves': 49, 'max_depth': 6, 'min_child_samples': 69, 'colsample_bytree': 0.6647299768033469, 'subsample': 0.7499523242435641, 'reg_lambda': 6.926376952329442, 'reg_alpha': 10.120382373657769}. Best is trial 1 with value: 1015.6236199327619.
[I 2025-03-17 19:29:25,297] Trial 2 finished with value: 1016.7322905198513 and parameters: {'learning_rate': 0.009612767520747835, 

[I 2025-03-17 19:29:29,915] Trial 22 finished with value: 1014.4235520481785 and parameters: {'learning_rate': 0.02587886572549046, 'num_leaves': 64, 'max_depth': 4, 'min_child_samples': 65, 'colsample_bytree': 0.8324005236637303, 'subsample': 0.7341608446498762, 'reg_lambda': 5.000360037383532, 'reg_alpha': 6.185967406954181}. Best is trial 15 with value: 1013.4773974672255.
[I 2025-03-17 19:29:30,048] Trial 23 finished with value: 1016.5194423632778 and parameters: {'learning_rate': 0.02269633501875821, 'num_leaves': 60, 'max_depth': 3, 'min_child_samples': 61, 'colsample_bytree': 0.8491606851840044, 'subsample': 0.764307372052811, 'reg_lambda': 6.359651060523591, 'reg_alpha': 5.0916408657331935}. Best is trial 15 with value: 1013.4773974672255.
[I 2025-03-17 19:29:30,175] Trial 24 finished with value: 1015.0664505247672 and parameters: {'learning_rate': 0.029530008030412735, 'num_leaves': 88, 'max_depth': 4, 'min_child_samples': 66, 'colsample_bytree': 0.7955430268630491, 'subsample

[I 2025-03-17 19:29:33,520] Trial 44 finished with value: 1013.6739604952787 and parameters: {'learning_rate': 0.010744378354682714, 'num_leaves': 100, 'max_depth': 4, 'min_child_samples': 60, 'colsample_bytree': 0.7381491703950612, 'subsample': 0.8296723320045064, 'reg_lambda': 5.616860699450754, 'reg_alpha': 8.17902606196397}. Best is trial 15 with value: 1013.4773974672255.
[I 2025-03-17 19:29:33,792] Trial 45 finished with value: 1015.9614566522367 and parameters: {'learning_rate': 0.009742171353162462, 'num_leaves': 100, 'max_depth': 5, 'min_child_samples': 60, 'colsample_bytree': 0.7149465941486752, 'subsample': 0.8657500243050565, 'reg_lambda': 6.6605554035343095, 'reg_alpha': 8.033561595495616}. Best is trial 15 with value: 1013.4773974672255.
[I 2025-03-17 19:29:34,042] Trial 46 finished with value: 1014.0983629589455 and parameters: {'learning_rate': 0.011505416319232258, 'num_leaves': 97, 'max_depth': 4, 'min_child_samples': 57, 'colsample_bytree': 0.7394355804772909, 'subsa

Best Parameters: {'learning_rate': 0.022843340298252773, 'num_leaves': 77, 'max_depth': 4, 'min_child_samples': 61, 'colsample_bytree': 0.8281427291607215, 'subsample': 0.765113035542883, 'reg_lambda': 6.880241474132783, 'reg_alpha': 7.438342652709071}


In [31]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Best parameters from Optuna
best_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.022843340298252773,  
    'num_leaves': 77,  # Slightly more complex trees
    'max_depth': 4,  
    'min_child_samples': 61,  # Allows a bit more flexibility in splits
    'subsample': 0.765113035542883,  
    'colsample_bytree': 0.8281427291607215,  
    'n_estimators': 3000,  
    'reg_lambda': 6.880241474132783,  # Reduce regularization slightly
    'reg_alpha': 7.438342652709071,  # Reduce regularization slightly
}



# Train model
model = LGBMRegressor(**best_params)
model.fit(X_train, y_train, 
          eval_set=[(X_validate, y_validate)], 
          eval_metric='rmse', 
          callbacks=[lgb.early_stopping(50, verbose=False)])

# Predict on training & validation sets
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_validate)

# Evaluate performance
def evaluate(y_true, y_pred, dataset):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n✅ Model Performance on {dataset}:")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R² Score: {r2:.4f}")

# Show results
evaluate(y_train, y_train_pred, "Training Set")
evaluate(y_validate, y_valid_pred, "Validation Set")

# Predict on test set
y_test_preds = model.predict(X_test)

# Create submission file
submission = test_df[['Item_Identifier', 'Outlet_Identifier']].copy()
submission['Item_Outlet_Sales'] = y_test_preds

submission['Item_Outlet_Sales'] = y_test_preds.clip(min=0)

# Save CSV
submission.to_csv('../submission/submission_lgb.csv', index=False)

print("Download your submission file: /mnt/data/submission_xgboost.csv")
#1151.2782174827 rank #986



✅ Model Performance on Training Set:
MAE: 746.0255
MSE: 1128399.2446
RMSE: 1062.2614
R² Score: 0.6185

✅ Model Performance on Validation Set:
MAE: 711.9326
MSE: 1027136.4352
RMSE: 1013.4774
R² Score: 0.6221
Download your submission file: /mnt/data/submission_xgboost.csv


In [46]:

import pandas as pd
import numpy as np
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Create a new combined categorical feature
train_df['Outlet_Combined'] = train_df['Outlet_Type'].astype(str) \
                                + "_" + train_df['Outlet_Size'].astype(str)
#train_df['Outlet_Location_Type'].astype(str) \
                              #+ "_" + \
                                

test_df['Outlet_Combined'] =    test_df['Outlet_Type'].astype(str) \
                                + "_" + test_df['Outlet_Size'].astype(str)
#test_df['Outlet_Location_Type'].astype(str) \
                              #+ "_" + \

train_df['Item_MRP_Per_Unit_Weight'] = train_df['Item_MRP']/train_df['Item_Weight']
test_df['Item_MRP_Per_Unit_Weight'] = test_df['Item_MRP']/test_df['Item_Weight']
    
    
# Select features and target
categorical_cols = [ 'Outlet_Combined']
numerical_cols = ['Item_MRP', 'Outlet_Total_Sales', 'Outlet_Age']

# Ensure categorical columns are treated as category dtype
for col in categorical_cols:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

# Select X and y
X = train_df[numerical_cols + categorical_cols]
y = train_df['Item_Outlet_Sales']
X_test = test_df[numerical_cols + categorical_cols]

# Split data
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.03),  # Wider range
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),  # More flexibility
        'max_depth': trial.suggest_int('max_depth', 3, 7),  # Slightly deeper trees
        'min_child_samples': trial.suggest_int('min_child_samples', 30, 70),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.85),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'reg_lambda': trial.suggest_float('reg_lambda', 5, 12),  # Stronger L2 regularization
        'reg_alpha': trial.suggest_float('reg_alpha', 5, 12),  # Stronger L1 regularization
        'n_estimators': 2000,  # Increased for better convergence
        'verbosity': -1  # Reduce logs for speed
    }

    model = lgb.LGBMRegressor(**params)

    eval_result = {}
    model.fit(
        X_train, y_train, 
        eval_set=[(X_validate, y_validate)], 
        eval_metric='rmse',
        categorical_feature=categorical_cols,  # Pass categorical features!
        callbacks=[lgb.early_stopping(100, verbose=False), lgb.record_evaluation(eval_result)]
    )
    
    preds = model.predict(X_validate)
    rmse = mean_squared_error(y_validate, preds, squared=False)
    return rmse

# Run Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  # Increase trials for better tuning

print("Best Parameters:", study.best_params)


[I 2025-03-17 23:17:51,459] A new study created in memory with name: no-name-bf07e4a4-8bd8-4356-9dc7-c56b1661c030
[I 2025-03-17 23:17:51,605] Trial 0 finished with value: 1019.9969747111944 and parameters: {'learning_rate': 0.025520738438971523, 'num_leaves': 97, 'max_depth': 6, 'min_child_samples': 37, 'colsample_bytree': 0.7208679212291934, 'subsample': 0.7036444249821978, 'reg_lambda': 7.379507860929351, 'reg_alpha': 9.545054071885456}. Best is trial 0 with value: 1019.9969747111944.
[I 2025-03-17 23:17:51,793] Trial 1 finished with value: 1017.0367688003355 and parameters: {'learning_rate': 0.007898004284722666, 'num_leaves': 42, 'max_depth': 3, 'min_child_samples': 46, 'colsample_bytree': 0.7844175633404089, 'subsample': 0.8575323899121057, 'reg_lambda': 10.482140895484118, 'reg_alpha': 10.297146610643315}. Best is trial 1 with value: 1017.0367688003355.
[I 2025-03-17 23:17:51,920] Trial 2 finished with value: 1016.5248514329436 and parameters: {'learning_rate': 0.0205480735140283

[I 2025-03-17 23:17:55,690] Trial 22 finished with value: 1016.438258817468 and parameters: {'learning_rate': 0.005317966528236167, 'num_leaves': 29, 'max_depth': 4, 'min_child_samples': 68, 'colsample_bytree': 0.8456280875089224, 'subsample': 0.8436469729881206, 'reg_lambda': 11.136965107541496, 'reg_alpha': 7.7615373335271745}. Best is trial 16 with value: 1016.3080023630292.
[I 2025-03-17 23:17:55,867] Trial 23 finished with value: 1017.8157317458954 and parameters: {'learning_rate': 0.016338067828764465, 'num_leaves': 26, 'max_depth': 5, 'min_child_samples': 69, 'colsample_bytree': 0.8486134003525411, 'subsample': 0.8587038448364743, 'reg_lambda': 10.1070359825578, 'reg_alpha': 7.7655817012585215}. Best is trial 16 with value: 1016.3080023630292.
[I 2025-03-17 23:17:56,111] Trial 24 finished with value: 1016.6064896379651 and parameters: {'learning_rate': 0.009509723383188732, 'num_leaves': 44, 'max_depth': 4, 'min_child_samples': 63, 'colsample_bytree': 0.8228467075758786, 'subsam

[I 2025-03-17 23:18:00,768] Trial 44 finished with value: 1018.7140616634932 and parameters: {'learning_rate': 0.01014598141277328, 'num_leaves': 49, 'max_depth': 5, 'min_child_samples': 68, 'colsample_bytree': 0.8326251367025296, 'subsample': 0.8232866541004148, 'reg_lambda': 11.37755941270404, 'reg_alpha': 8.260635501232146}. Best is trial 16 with value: 1016.3080023630292.
[I 2025-03-17 23:18:00,996] Trial 45 finished with value: 1016.7416722653154 and parameters: {'learning_rate': 0.008567151497347433, 'num_leaves': 23, 'max_depth': 4, 'min_child_samples': 61, 'colsample_bytree': 0.7905717866962261, 'subsample': 0.8934790296134133, 'reg_lambda': 10.40995192052265, 'reg_alpha': 9.119253017803517}. Best is trial 16 with value: 1016.3080023630292.
[I 2025-03-17 23:18:01,155] Trial 46 finished with value: 1017.1358296912608 and parameters: {'learning_rate': 0.01626233018777472, 'num_leaves': 29, 'max_depth': 3, 'min_child_samples': 58, 'colsample_bytree': 0.809834303512462, 'subsample'

Best Parameters: {'learning_rate': 0.011665974909561828, 'num_leaves': 40, 'max_depth': 4, 'min_child_samples': 66, 'colsample_bytree': 0.8042619634209697, 'subsample': 0.8447182011205896, 'reg_lambda': 9.674946890801323, 'reg_alpha': 8.352998066662282}


In [41]:
study.best_params

{'learning_rate': 0.007974090162682806,
 'num_leaves': 68,
 'max_depth': 4,
 'min_child_samples': 60,
 'colsample_bytree': 0.6839413130786991,
 'subsample': 0.8318953425430651,
 'reg_lambda': 10.422835186967905,
 'reg_alpha': 7.573546409151058}

In [47]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Best parameters from Optuna
best_params = study.best_params.copy()  # Copy the original dictionary
best_params.update({  
    'objective': 'regression',  
    'metric': 'rmse',  
    'n_estimators': 2000  
})



# Train model
model = LGBMRegressor(**best_params)
model.fit(X_train, y_train, 
          eval_set=[(X_validate, y_validate)], 
          eval_metric='rmse', 
          callbacks=[lgb.early_stopping(50, verbose=False)])

# Predict on training & validation sets
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_validate)

# Evaluate performance
def evaluate(y_true, y_pred, dataset):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n✅ Model Performance on {dataset}:")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R² Score: {r2:.4f}")

# Show results
evaluate(y_train, y_train_pred, "Training Set")
evaluate(y_validate, y_valid_pred, "Validation Set")

# Predict on test set
y_test_preds = model.predict(X_test)

# Create submission file
submission = test_df[['Item_Identifier', 'Outlet_Identifier']].copy()
submission['Item_Outlet_Sales'] = y_test_preds

submission['Item_Outlet_Sales'] = y_test_preds.clip(min=0)

# Save CSV
submission.to_csv('../submission/submission_lgb.csv', index=False)

print("Download your submission file: /mnt/data/submission_xgboost.csv")
#1150.9645886406 Rank #931



✅ Model Performance on Training Set:
MAE: 755.3883
MSE: 1152884.3301
RMSE: 1073.7245
R² Score: 0.6103

✅ Model Performance on Validation Set:
MAE: 714.2423
MSE: 1032881.9557
RMSE: 1016.3080
R² Score: 0.6200
Download your submission file: /mnt/data/submission_xgboost.csv


In [80]:

import pandas as pd
import numpy as np
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Define age bins (including 0 properly)
bins = [-1, 10, 20, float('inf')]  # -1 ensures 0 is included in "New"
labels = ['New', 'Moderate', 'Old']

# Apply binning
train_df['Outlet_Age_Binned'] = pd.cut(train_df['Outlet_Age'], bins=bins, labels=labels)
# Apply the same binning to test_df
test_df['Outlet_Age_Binned'] = pd.cut(test_df['Outlet_Age'], bins=bins, labels=labels)



# Create a new combined categorical feature
train_df['Outlet_Combined'] = train_df['Outlet_Type'].astype(str) \
+ "_" + train_df['Outlet_Size'].astype(str)
#+ "_" + train_df['Outlet_Age_Binned'].astype(str)
#+ "_" + train_df['Outlet_Location_Type'].astype(str)
                              
                                

test_df['Outlet_Combined'] =    test_df['Outlet_Type'].astype(str) \
+ "_" + test_df['Outlet_Size'].astype(str)
#+ "_" + test_df['Outlet_Age_Binned'].astype(str)
#+ "_" + test_df['Outlet_Location_Type'].astype(str)


train_df['Item_MRP_Per_Unit_Weight'] = train_df['Item_MRP']/train_df['Item_Weight']
test_df['Item_MRP_Per_Unit_Weight'] = test_df['Item_MRP']/test_df['Item_Weight']
    
    
# Select features and target
categorical_cols = [ 'Outlet_Combined']
numerical_cols = ['Item_MRP', 'Outlet_Total_Sales', 'Item_Visibility', 'Outlet_Age' ]

# Ensure categorical columns are treated as category dtype
for col in categorical_cols:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

# Select X and y
X = train_df[numerical_cols + categorical_cols]
y = train_df['Item_Outlet_Sales']
X_test = test_df[numerical_cols + categorical_cols]

# Split data
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.03),  # Wider range
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),  # More flexibility
        'max_depth': trial.suggest_int('max_depth', 3, 7),  # Slightly deeper trees
        'min_child_samples': trial.suggest_int('min_child_samples', 30, 70),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.85),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'reg_lambda': trial.suggest_float('reg_lambda', 5, 12),  # Stronger L2 regularization
        'reg_alpha': trial.suggest_float('reg_alpha', 5, 12),  # Stronger L1 regularization
        'n_estimators': 2000,  # Increased for better convergence
        'verbosity': -1  # Reduce logs for speed
    }

    model = lgb.LGBMRegressor(**params)

    eval_result = {}
    model.fit(
        X_train, y_train, 
        eval_set=[(X_validate, y_validate)], 
        eval_metric='rmse',
        categorical_feature=categorical_cols,  # Pass categorical features!
        callbacks=[lgb.early_stopping(100, verbose=False), lgb.record_evaluation(eval_result)]
    )
    
    preds = model.predict(X_validate)
    rmse = mean_squared_error(y_validate, preds, squared=False)
    return rmse

# Run Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  # Increase trials for better tuning

print("Best Parameters:", study.best_params)


[I 2025-03-17 23:46:30,176] A new study created in memory with name: no-name-b6b9ed56-ffd0-4399-8937-c4f7eb278ded
[I 2025-03-17 23:46:30,373] Trial 0 finished with value: 1013.8863422536131 and parameters: {'learning_rate': 0.023312650180224208, 'num_leaves': 26, 'max_depth': 7, 'min_child_samples': 51, 'colsample_bytree': 0.8305800829393507, 'subsample': 0.7957933426890674, 'reg_lambda': 11.077838440567294, 'reg_alpha': 11.629281257572512}. Best is trial 0 with value: 1013.8863422536131.
[I 2025-03-17 23:46:30,516] Trial 1 finished with value: 1016.6531742166126 and parameters: {'learning_rate': 0.028455559321691824, 'num_leaves': 88, 'max_depth': 5, 'min_child_samples': 63, 'colsample_bytree': 0.6530460300667424, 'subsample': 0.8736825121214028, 'reg_lambda': 6.734001183222088, 'reg_alpha': 7.97278012006481}. Best is trial 0 with value: 1013.8863422536131.
[I 2025-03-17 23:46:30,797] Trial 2 finished with value: 1014.2798108652606 and parameters: {'learning_rate': 0.01252891998451368

[I 2025-03-17 23:46:34,936] Trial 22 finished with value: 1014.5501707597149 and parameters: {'learning_rate': 0.029849281930100446, 'num_leaves': 29, 'max_depth': 6, 'min_child_samples': 40, 'colsample_bytree': 0.8256056370610484, 'subsample': 0.7767759719588074, 'reg_lambda': 11.5439573386696, 'reg_alpha': 9.845293195651141}. Best is trial 11 with value: 1012.3248369436131.
[I 2025-03-17 23:46:35,105] Trial 23 finished with value: 1012.2780147061791 and parameters: {'learning_rate': 0.026928692206680278, 'num_leaves': 21, 'max_depth': 7, 'min_child_samples': 48, 'colsample_bytree': 0.7819276372978132, 'subsample': 0.745248666361766, 'reg_lambda': 10.590042512887734, 'reg_alpha': 8.518583571926653}. Best is trial 23 with value: 1012.2780147061791.
[I 2025-03-17 23:46:35,307] Trial 24 finished with value: 1013.4762347342282 and parameters: {'learning_rate': 0.02130178158430693, 'num_leaves': 21, 'max_depth': 7, 'min_child_samples': 48, 'colsample_bytree': 0.7410672389097448, 'subsample

[I 2025-03-17 23:46:39,689] Trial 44 finished with value: 1016.0096916791596 and parameters: {'learning_rate': 0.02414736544531328, 'num_leaves': 30, 'max_depth': 7, 'min_child_samples': 41, 'colsample_bytree': 0.7090134100970663, 'subsample': 0.7366121638198301, 'reg_lambda': 11.702088281494126, 'reg_alpha': 9.509710754399512}. Best is trial 23 with value: 1012.2780147061791.
[I 2025-03-17 23:46:39,896] Trial 45 finished with value: 1013.2729879309178 and parameters: {'learning_rate': 0.0251473438330294, 'num_leaves': 24, 'max_depth': 6, 'min_child_samples': 43, 'colsample_bytree': 0.7254131851439182, 'subsample': 0.7273582555654383, 'reg_lambda': 5.002192413175663, 'reg_alpha': 10.474990599199199}. Best is trial 23 with value: 1012.2780147061791.
[I 2025-03-17 23:46:40,240] Trial 46 finished with value: 1019.5715405349076 and parameters: {'learning_rate': 0.013710834801559369, 'num_leaves': 28, 'max_depth': 7, 'min_child_samples': 61, 'colsample_bytree': 0.667037497894872, 'subsample

Best Parameters: {'learning_rate': 0.026928692206680278, 'num_leaves': 21, 'max_depth': 7, 'min_child_samples': 48, 'colsample_bytree': 0.7819276372978132, 'subsample': 0.745248666361766, 'reg_lambda': 10.590042512887734, 'reg_alpha': 8.518583571926653}


In [79]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Best parameters from Optuna
best_params = study.best_params.copy()  # Copy the original dictionary
best_params.update({  
    'objective': 'regression',  
    'metric': 'rmse',  
    'n_estimators': 2000  
})



# Train model
model = LGBMRegressor(**best_params)
model.fit(X_train, y_train, 
          eval_set=[(X_validate, y_validate)], 
          eval_metric='rmse', 
          callbacks=[lgb.early_stopping(50, verbose=False)])

# Predict on training & validation sets
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_validate)

# Evaluate performance
def evaluate(y_true, y_pred, dataset):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n✅ Model Performance on {dataset}:")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R² Score: {r2:.4f}")

# Show results
evaluate(y_train, y_train_pred, "Training Set")
evaluate(y_validate, y_valid_pred, "Validation Set")

# Predict on test set
y_test_preds = model.predict(X_test)

# Create submission file
submission = test_df[['Item_Identifier', 'Outlet_Identifier']].copy()
submission['Item_Outlet_Sales'] = y_test_preds

submission['Item_Outlet_Sales'] = y_test_preds.clip(min=0)

# Save CSV
submission.to_csv('../submission/submission_lgb.csv', index=False)

print("Download your submission file: /mnt/data/submission_xgboost.csv")



✅ Model Performance on Training Set:
MAE: 739.7439
MSE: 1111898.5934
RMSE: 1054.4660
R² Score: 0.6241

✅ Model Performance on Validation Set:
MAE: 711.4432
MSE: 1025056.9816
RMSE: 1012.4510
R² Score: 0.6229
Download your submission file: /mnt/data/submission_xgboost.csv


In [95]:

import pandas as pd
import numpy as np
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Define age bins (including 0 properly)
bins = [-1, 10, 20, float('inf')]  # -1 ensures 0 is included in "New"
labels = ['New', 'Moderate', 'Old']

# Apply binning
train_df['Outlet_Age_Binned'] = pd.cut(train_df['Outlet_Age'], bins=bins, labels=labels)
# Apply the same binning to test_df
test_df['Outlet_Age_Binned'] = pd.cut(test_df['Outlet_Age'], bins=bins, labels=labels)



# Create a new combined categorical feature
train_df['Outlet_Combined'] = train_df['Outlet_Size'].astype(str) \
+ "_" + train_df['Outlet_Type'].astype(str)
#+ "_" + train_df['Outlet_Age_Binned'].astype(str)
#+ "_" + train_df['Outlet_Location_Type'].astype(str)
                              
                                

test_df['Outlet_Combined'] =    test_df['Outlet_Size'].astype(str) \
+ "_" + test_df['Outlet_Type'].astype(str)
#+ "_" + test_df['Outlet_Age_Binned'].astype(str)
#+ "_" + test_df['Outlet_Location_Type'].astype(str)


train_df['Item_MRP_Per_Unit_Weight'] = train_df['Item_MRP']/train_df['Item_Weight']
test_df['Item_MRP_Per_Unit_Weight'] = test_df['Item_MRP']/test_df['Item_Weight']
    
    
# Select features and target
categorical_cols = [  'Outlet_Combined', ]
numerical_cols = ['Item_MRP','Outlet_Total_Sales' ]

# Ensure categorical columns are treated as category dtype
for col in categorical_cols:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

# Select X and y
X = train_df[numerical_cols + categorical_cols]
y = train_df['Item_Outlet_Sales']
X_test = test_df[numerical_cols + categorical_cols]

# Split data
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.03),  # Wider range
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),  # More flexibility
        'max_depth': trial.suggest_int('max_depth', 3, 7),  # Slightly deeper trees
        'min_child_samples': trial.suggest_int('min_child_samples', 30, 70),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.85),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'reg_lambda': trial.suggest_float('reg_lambda', 5, 12),  # Stronger L2 regularization
        'reg_alpha': trial.suggest_float('reg_alpha', 5, 12),  # Stronger L1 regularization
        'n_estimators': 2000,  # Increased for better convergence
        'verbosity': -1  # Reduce logs for speed
    }

    model = lgb.LGBMRegressor(**params)

    eval_result = {}
    model.fit(
        X_train, y_train, 
        eval_set=[(X_validate, y_validate)], 
        eval_metric='rmse',
        categorical_feature=categorical_cols,  # Pass categorical features!
        callbacks=[lgb.early_stopping(100, verbose=False), lgb.record_evaluation(eval_result)]
    )
    
    preds = model.predict(X_validate)
    rmse = mean_squared_error(y_validate, preds, squared=False)
    return rmse

# Run Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  # Increase trials for better tuning

print("Best Parameters:", study.best_params)


[I 2025-03-18 00:09:21,132] A new study created in memory with name: no-name-23422226-80b2-462f-ba0f-5fa752d43896
[I 2025-03-18 00:09:21,453] Trial 0 finished with value: 1021.1271965497546 and parameters: {'learning_rate': 0.014497099121579565, 'num_leaves': 62, 'max_depth': 6, 'min_child_samples': 32, 'colsample_bytree': 0.631051077575546, 'subsample': 0.8432251483502375, 'reg_lambda': 5.490493121960747, 'reg_alpha': 6.8638947336909935}. Best is trial 0 with value: 1021.1271965497546.
[I 2025-03-18 00:09:21,648] Trial 1 finished with value: 1018.4864503367753 and parameters: {'learning_rate': 0.013496828786359136, 'num_leaves': 67, 'max_depth': 6, 'min_child_samples': 66, 'colsample_bytree': 0.8373494050774316, 'subsample': 0.8554168805210023, 'reg_lambda': 8.796965232346773, 'reg_alpha': 6.908922762180443}. Best is trial 1 with value: 1018.4864503367753.
[I 2025-03-18 00:09:21,813] Trial 2 finished with value: 1017.9183067196623 and parameters: {'learning_rate': 0.028360819335070418

[I 2025-03-18 00:09:26,649] Trial 22 finished with value: 1016.6003162428235 and parameters: {'learning_rate': 0.02330403431223816, 'num_leaves': 43, 'max_depth': 3, 'min_child_samples': 35, 'colsample_bytree': 0.7296885162701975, 'subsample': 0.7908850998144753, 'reg_lambda': 11.128629303805079, 'reg_alpha': 9.894746052661041}. Best is trial 12 with value: 1016.4778142873837.
[I 2025-03-18 00:09:26,820] Trial 23 finished with value: 1018.3288410339326 and parameters: {'learning_rate': 0.025088013541285176, 'num_leaves': 42, 'max_depth': 4, 'min_child_samples': 30, 'colsample_bytree': 0.7902876801387441, 'subsample': 0.7824394673663031, 'reg_lambda': 11.248884823375217, 'reg_alpha': 9.97987904727653}. Best is trial 12 with value: 1016.4778142873837.
[I 2025-03-18 00:09:27,048] Trial 24 finished with value: 1016.6129186051228 and parameters: {'learning_rate': 0.017153617366572477, 'num_leaves': 53, 'max_depth': 3, 'min_child_samples': 34, 'colsample_bytree': 0.7523675598197134, 'subsamp

[I 2025-03-18 00:09:32,210] Trial 44 finished with value: 1016.5340451162508 and parameters: {'learning_rate': 0.024456574341018222, 'num_leaves': 40, 'max_depth': 3, 'min_child_samples': 30, 'colsample_bytree': 0.691410434189735, 'subsample': 0.7880861041505499, 'reg_lambda': 11.487721624624102, 'reg_alpha': 10.499562523488294}. Best is trial 12 with value: 1016.4778142873837.
[I 2025-03-18 00:09:32,399] Trial 45 finished with value: 1018.9867624124903 and parameters: {'learning_rate': 0.028072362623674033, 'num_leaves': 41, 'max_depth': 4, 'min_child_samples': 33, 'colsample_bytree': 0.6829442186987819, 'subsample': 0.7621207529458582, 'reg_lambda': 11.61574829707777, 'reg_alpha': 6.407408384429846}. Best is trial 12 with value: 1016.4778142873837.
[I 2025-03-18 00:09:32,593] Trial 46 finished with value: 1018.5490114221017 and parameters: {'learning_rate': 0.02625878523137902, 'num_leaves': 62, 'max_depth': 4, 'min_child_samples': 36, 'colsample_bytree': 0.7023225290447428, 'subsamp

Best Parameters: {'learning_rate': 0.0214956903520105, 'num_leaves': 29, 'max_depth': 3, 'min_child_samples': 34, 'colsample_bytree': 0.6799166532295509, 'subsample': 0.7692353996939044, 'reg_lambda': 10.326012198865957, 'reg_alpha': 10.22346257287167}


In [96]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Best parameters from Optuna
best_params = study.best_params.copy()  # Copy the original dictionary
best_params.update({  
    'objective': 'regression',  
    'metric': 'rmse',  
    'n_estimators': 2000  
})



# Train model
model = LGBMRegressor(**best_params)
model.fit(X_train, y_train, 
          eval_set=[(X_validate, y_validate)], 
          eval_metric='rmse', 
          callbacks=[lgb.early_stopping(50, verbose=False)])

# Predict on training & validation sets
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_validate)

# Evaluate performance
def evaluate(y_true, y_pred, dataset):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n✅ Model Performance on {dataset}:")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R² Score: {r2:.4f}")

# Show results
evaluate(y_train, y_train_pred, "Training Set")
evaluate(y_validate, y_valid_pred, "Validation Set")

# Predict on test set
y_test_preds = model.predict(X_test)

# Create submission file
submission = test_df[['Item_Identifier', 'Outlet_Identifier']].copy()
submission['Item_Outlet_Sales'] = y_test_preds

submission['Item_Outlet_Sales'] = y_test_preds.clip(min=0)

# Save CSV
submission.to_csv('../submission/submission_lgb.csv', index=False)

print("Download your submission file: /mnt/data/submission_xgboost.csv")



✅ Model Performance on Training Set:
MAE: 758.0353
MSE: 1157039.9712
RMSE: 1075.6579
R² Score: 0.6089

✅ Model Performance on Validation Set:
MAE: 715.8103
MSE: 1033227.1469
RMSE: 1016.4778
R² Score: 0.6199
Download your submission file: /mnt/data/submission_xgboost.csv


In [98]:

import pandas as pd
import numpy as np
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Create a new combined categorical feature
train_df['Outlet_Combined'] = train_df['Outlet_Type'].astype(str) \
                                + "_" + train_df['Outlet_Size'].astype(str)
#train_df['Outlet_Location_Type'].astype(str) \
                              #+ "_" + \
                                

test_df['Outlet_Combined'] =    test_df['Outlet_Type'].astype(str) \
                                + "_" + test_df['Outlet_Size'].astype(str)
#test_df['Outlet_Location_Type'].astype(str) \
                              #+ "_" + \

train_df['Item_MRP_Per_Unit_Weight'] = train_df['Item_MRP']/train_df['Item_Weight']
test_df['Item_MRP_Per_Unit_Weight'] = test_df['Item_MRP']/test_df['Item_Weight']

Outlet_Sales_Mean = train_df.groupby('Outlet_Identifier')['Item_Outlet_Sales'].mean()
train_df['Outlet_Sales_Mean'] = train_df['Outlet_Identifier'].map(Outlet_Sales_Mean)
test_df['Outlet_Sales_Mean'] = test_df['Outlet_Identifier'].map(Outlet_Sales_Mean)

    
# Select features and target
categorical_cols = [ 'Outlet_Combined']
numerical_cols = ['Item_MRP', 'Outlet_Sales_Mean', 'Outlet_Age']

# Ensure categorical columns are treated as category dtype
for col in categorical_cols:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

# Select X and y
X = train_df[numerical_cols + categorical_cols]
y = train_df['Item_Outlet_Sales']
X_test = test_df[numerical_cols + categorical_cols]

# Split data
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.03),  # Wider range
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),  # More flexibility
        'max_depth': trial.suggest_int('max_depth', 3, 7),  # Slightly deeper trees
        'min_child_samples': trial.suggest_int('min_child_samples', 30, 70),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.85),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'reg_lambda': trial.suggest_float('reg_lambda', 5, 12),  # Stronger L2 regularization
        'reg_alpha': trial.suggest_float('reg_alpha', 5, 12),  # Stronger L1 regularization
        'n_estimators': 2000,  # Increased for better convergence
        'verbosity': -1  # Reduce logs for speed
    }

    model = lgb.LGBMRegressor(**params)

    eval_result = {}
    model.fit(
        X_train, y_train, 
        eval_set=[(X_validate, y_validate)], 
        eval_metric='rmse',
        categorical_feature=categorical_cols,  # Pass categorical features!
        callbacks=[lgb.early_stopping(100, verbose=False), lgb.record_evaluation(eval_result)]
    )
    
    preds = model.predict(X_validate)
    rmse = mean_squared_error(y_validate, preds, squared=False)
    return rmse

# Run Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  # Increase trials for better tuning

print("Best Parameters:", study.best_params)


[I 2025-03-18 00:19:00,390] A new study created in memory with name: no-name-bed3b0d6-f0eb-4ed4-8944-204d7dc385a2
[I 2025-03-18 00:19:00,536] Trial 0 finished with value: 1018.3713719756884 and parameters: {'learning_rate': 0.016305293441401962, 'num_leaves': 58, 'max_depth': 4, 'min_child_samples': 69, 'colsample_bytree': 0.6345682367086382, 'subsample': 0.7361808520073196, 'reg_lambda': 10.344080875299174, 'reg_alpha': 7.438539052577884}. Best is trial 0 with value: 1018.3713719756884.
[I 2025-03-18 00:19:00,694] Trial 1 finished with value: 1023.5569342394767 and parameters: {'learning_rate': 0.022096043350414887, 'num_leaves': 45, 'max_depth': 7, 'min_child_samples': 52, 'colsample_bytree': 0.7423174068718611, 'subsample': 0.749444167628759, 'reg_lambda': 7.978629498371472, 'reg_alpha': 8.298437392429168}. Best is trial 0 with value: 1018.3713719756884.
[I 2025-03-18 00:19:00,938] Trial 2 finished with value: 1018.7204004912826 and parameters: {'learning_rate': 0.009629845505575732

[I 2025-03-18 00:19:05,779] Trial 22 finished with value: 1018.1805981356682 and parameters: {'learning_rate': 0.007608023053262707, 'num_leaves': 44, 'max_depth': 3, 'min_child_samples': 65, 'colsample_bytree': 0.6534777905487521, 'subsample': 0.8181421717289074, 'reg_lambda': 7.322960285230865, 'reg_alpha': 9.780361486512692}. Best is trial 5 with value: 1017.8943266052809.
[I 2025-03-18 00:19:05,997] Trial 23 finished with value: 1018.522363195462 and parameters: {'learning_rate': 0.011443127957367754, 'num_leaves': 31, 'max_depth': 4, 'min_child_samples': 59, 'colsample_bytree': 0.6343777238237003, 'subsample': 0.8399026583946707, 'reg_lambda': 5.915883746956855, 'reg_alpha': 11.077927483710086}. Best is trial 5 with value: 1017.8943266052809.
[I 2025-03-18 00:19:06,192] Trial 24 finished with value: 1018.1279032852259 and parameters: {'learning_rate': 0.014767688128621772, 'num_leaves': 26, 'max_depth': 3, 'min_child_samples': 66, 'colsample_bytree': 0.7039140804570116, 'subsample

[I 2025-03-18 00:19:10,715] Trial 44 finished with value: 1017.8872646932706 and parameters: {'learning_rate': 0.010835947599748093, 'num_leaves': 56, 'max_depth': 3, 'min_child_samples': 68, 'colsample_bytree': 0.7091276247457078, 'subsample': 0.7269998270417688, 'reg_lambda': 7.447288163932868, 'reg_alpha': 6.461940046055087}. Best is trial 33 with value: 1017.7862845678826.
[I 2025-03-18 00:19:10,909] Trial 45 finished with value: 1018.2317210314893 and parameters: {'learning_rate': 0.01399116290566624, 'num_leaves': 73, 'max_depth': 3, 'min_child_samples': 68, 'colsample_bytree': 0.7135442577653077, 'subsample': 0.7255563570667785, 'reg_lambda': 7.477650842546769, 'reg_alpha': 6.482544428038504}. Best is trial 33 with value: 1017.7862845678826.
[I 2025-03-18 00:19:11,300] Trial 46 finished with value: 1021.4136009949879 and parameters: {'learning_rate': 0.008632595956419816, 'num_leaves': 62, 'max_depth': 6, 'min_child_samples': 39, 'colsample_bytree': 0.7527757856242238, 'subsampl

Best Parameters: {'learning_rate': 0.01112450477916466, 'num_leaves': 47, 'max_depth': 3, 'min_child_samples': 68, 'colsample_bytree': 0.8047939619276595, 'subsample': 0.7511958570053101, 'reg_lambda': 7.5278930435842, 'reg_alpha': 6.307137354889451}


In [99]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Best parameters from Optuna
best_params = study.best_params.copy()  # Copy the original dictionary
best_params.update({  
    'objective': 'regression',  
    'metric': 'rmse',  
    'n_estimators': 2000  
})



# Train model
model = LGBMRegressor(**best_params)
model.fit(X_train, y_train, 
          eval_set=[(X_validate, y_validate)], 
          eval_metric='rmse', 
          callbacks=[lgb.early_stopping(50, verbose=False)])

# Predict on training & validation sets
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_validate)

# Evaluate performance
def evaluate(y_true, y_pred, dataset):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n✅ Model Performance on {dataset}:")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R² Score: {r2:.4f}")

# Show results
evaluate(y_train, y_train_pred, "Training Set")
evaluate(y_validate, y_valid_pred, "Validation Set")

# Predict on test set
y_test_preds = model.predict(X_test)

# Create submission file
submission = test_df[['Item_Identifier', 'Outlet_Identifier']].copy()
submission['Item_Outlet_Sales'] = y_test_preds

submission['Item_Outlet_Sales'] = y_test_preds.clip(min=0)

# Save CSV
submission.to_csv('../submission/submission_lgb.csv', index=False)

print("Download your submission file: /mnt/data/submission_xgboost.csv")



✅ Model Performance on Training Set:
MAE: 756.3971
MSE: 1155221.6873
RMSE: 1074.8124
R² Score: 0.6095

✅ Model Performance on Validation Set:
MAE: 715.9820
MSE: 1035115.7040
RMSE: 1017.4064
R² Score: 0.6192
Download your submission file: /mnt/data/submission_xgboost.csv


In [102]:
import pandas as pd
import numpy as np
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Create a new combined categorical feature
train_df['Outlet_Combined'] = train_df['Outlet_Type'].astype(str) \
                                + "_" + train_df['Outlet_Size'].astype(str)
#train_df['Outlet_Location_Type'].astype(str) \
                              #+ "_" + \
                                

test_df['Outlet_Combined'] =    test_df['Outlet_Type'].astype(str) \
                                + "_" + test_df['Outlet_Size'].astype(str)
#test_df['Outlet_Location_Type'].astype(str) \
                              #+ "_" + \

train_df['Item_MRP_Per_Unit_Weight'] = train_df['Item_MRP']/train_df['Item_Weight']
test_df['Item_MRP_Per_Unit_Weight'] = test_df['Item_MRP']/test_df['Item_Weight']
    
    
# Select features and target
categorical_cols = [ 'Outlet_Combined']
numerical_cols = ['Item_MRP', 'Outlet_Total_Sales', 'Outlet_Age']

# Ensure categorical columns are treated as category dtype
for col in categorical_cols:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

# Select X and y
X = train_df[numerical_cols + categorical_cols]
y = train_df['Item_Outlet_Sales']
X_test = test_df[numerical_cols + categorical_cols]

# Split data
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.03),  # Wider range
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),  # More flexibility
        'max_depth': trial.suggest_int('max_depth', 3, 7),  # Slightly deeper trees
        'min_child_samples': trial.suggest_int('min_child_samples', 30, 70),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.85),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'reg_lambda': trial.suggest_float('reg_lambda', 5, 12),  # Stronger L2 regularization
        'reg_alpha': trial.suggest_float('reg_alpha', 5, 12),  # Stronger L1 regularization
        'n_estimators': 3000,  # Increased for better convergence
        'verbosity': -1,  # Reduce logs for speed
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 50),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 0.9),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 0.9),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10)
    }

    model = lgb.LGBMRegressor(**params)

    eval_result = {}
    model.fit(
        X_train, y_train, 
        eval_set=[(X_validate, y_validate)], 
        eval_metric='rmse',
        categorical_feature=categorical_cols,  # Pass categorical features!
        callbacks=[lgb.early_stopping(100, verbose=False), lgb.record_evaluation(eval_result)]
    )
    
    preds = model.predict(X_validate)
    rmse = mean_squared_error(y_validate, preds, squared=False)
    return rmse

# Run Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  # Increase trials for better tuning

print("Best Parameters:", study.best_params)


[I 2025-03-18 00:32:29,683] A new study created in memory with name: no-name-1558c699-da4d-4f66-b5c7-8a06352cfe8d
[I 2025-03-18 00:32:31,099] Trial 0 finished with value: 1017.2518599090918 and parameters: {'learning_rate': 0.008976101143419974, 'num_leaves': 69, 'max_depth': 5, 'min_child_samples': 37, 'colsample_bytree': 0.6684982089962654, 'subsample': 0.8732247213179906, 'reg_lambda': 7.509068985363829, 'reg_alpha': 11.675302461207481, 'min_data_in_leaf': 37, 'feature_fraction': 0.7496719981360618, 'bagging_fraction': 0.7289691758001717, 'bagging_freq': 5}. Best is trial 0 with value: 1017.2518599090918.
[I 2025-03-18 00:32:31,723] Trial 1 finished with value: 1018.5305828836014 and parameters: {'learning_rate': 0.013426828801937216, 'num_leaves': 32, 'max_depth': 3, 'min_child_samples': 62, 'colsample_bytree': 0.6929484863618205, 'subsample': 0.7146613122553258, 'reg_lambda': 5.971114023196622, 'reg_alpha': 8.345636118243068, 'min_data_in_leaf': 50, 'feature_fraction': 0.554976435

[I 2025-03-18 00:32:42,036] Trial 17 finished with value: 1015.993584484429 and parameters: {'learning_rate': 0.012528019018013476, 'num_leaves': 100, 'max_depth': 3, 'min_child_samples': 49, 'colsample_bytree': 0.6772204485190181, 'subsample': 0.7957538118743296, 'reg_lambda': 10.602066552915772, 'reg_alpha': 6.0684272960501735, 'min_data_in_leaf': 40, 'feature_fraction': 0.817961420086961, 'bagging_fraction': 0.435144363615096, 'bagging_freq': 4}. Best is trial 17 with value: 1015.993584484429.
[I 2025-03-18 00:32:42,505] Trial 18 finished with value: 1016.2872563067546 and parameters: {'learning_rate': 0.01191407449359935, 'num_leaves': 99, 'max_depth': 3, 'min_child_samples': 51, 'colsample_bytree': 0.7572482531101908, 'subsample': 0.7903805545820425, 'reg_lambda': 10.794122073054698, 'reg_alpha': 6.082574670622332, 'min_data_in_leaf': 43, 'feature_fraction': 0.8345202932041105, 'bagging_fraction': 0.41084911667054286, 'bagging_freq': 3}. Best is trial 17 with value: 1015.993584484

[I 2025-03-18 00:32:49,584] Trial 34 finished with value: 1017.8591672748385 and parameters: {'learning_rate': 0.024424754069456697, 'num_leaves': 83, 'max_depth': 5, 'min_child_samples': 39, 'colsample_bytree': 0.8365314242735327, 'subsample': 0.7638910508783491, 'reg_lambda': 8.255373611215571, 'reg_alpha': 9.641143849148243, 'min_data_in_leaf': 37, 'feature_fraction': 0.7476865312616355, 'bagging_fraction': 0.5305922662626651, 'bagging_freq': 4}. Best is trial 29 with value: 1014.725848225669.
[I 2025-03-18 00:32:50,149] Trial 35 finished with value: 1016.5187341951267 and parameters: {'learning_rate': 0.021609611824044368, 'num_leaves': 77, 'max_depth': 6, 'min_child_samples': 34, 'colsample_bytree': 0.8210440258219694, 'subsample': 0.746483660986114, 'reg_lambda': 6.293673271201009, 'reg_alpha': 8.23036252849243, 'min_data_in_leaf': 49, 'feature_fraction': 0.8604583736403055, 'bagging_fraction': 0.4521076699531965, 'bagging_freq': 6}. Best is trial 29 with value: 1014.725848225669

Best Parameters: {'learning_rate': 0.02796894428379391, 'num_leaves': 62, 'max_depth': 3, 'min_child_samples': 37, 'colsample_bytree': 0.8314750865217698, 'subsample': 0.7735485016722371, 'reg_lambda': 11.373906300512306, 'reg_alpha': 7.089588914108556, 'min_data_in_leaf': 29, 'feature_fraction': 0.8008771789769791, 'bagging_fraction': 0.4198048096703424, 'bagging_freq': 6}


In [103]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Best parameters from Optuna
best_params = study.best_params.copy()  # Copy the original dictionary
best_params.update({  
    'objective': 'regression',  
    'metric': 'rmse',  
    'n_estimators': 2000  
})



# Train model
model = LGBMRegressor(**best_params)
model.fit(X_train, y_train, 
          eval_set=[(X_validate, y_validate)], 
          eval_metric='rmse', 
          callbacks=[lgb.early_stopping(50, verbose=False)])

# Predict on training & validation sets
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_validate)

# Evaluate performance
def evaluate(y_true, y_pred, dataset):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n✅ Model Performance on {dataset}:")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R² Score: {r2:.4f}")

# Show results
evaluate(y_train, y_train_pred, "Training Set")
evaluate(y_validate, y_valid_pred, "Validation Set")

# Predict on test set
y_test_preds = model.predict(X_test)

# Create submission file
submission = test_df[['Item_Identifier', 'Outlet_Identifier']].copy()
submission['Item_Outlet_Sales'] = y_test_preds

submission['Item_Outlet_Sales'] = y_test_preds.clip(min=0)

# Save CSV
submission.to_csv('../submission/submission_lgb.csv', index=False)

print("Download your submission file: /mnt/data/submission_xgboost.csv")
#1150.5794325247 Rank #831



✅ Model Performance on Training Set:
MAE: 762.7064
MSE: 1164127.5549
RMSE: 1078.9474
R² Score: 0.6065

✅ Model Performance on Validation Set:
MAE: 717.4974
MSE: 1031873.7988
RMSE: 1015.8119
R² Score: 0.6204
Download your submission file: /mnt/data/submission_xgboost.csv


In [108]:
import pandas as pd
import numpy as np
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Create a new combined categorical feature
train_df['Outlet_Combined'] = train_df['Outlet_Type'].astype(str) \
                                + "_" + train_df['Outlet_Size'].astype(str)
#train_df['Outlet_Location_Type'].astype(str) \
                              #+ "_" + \
                                

test_df['Outlet_Combined'] =    test_df['Outlet_Type'].astype(str) \
                                + "_" + test_df['Outlet_Size'].astype(str)
#test_df['Outlet_Location_Type'].astype(str) \
                              #+ "_" + \

train_df['Item_MRP_Per_Unit_Weight'] = train_df['Item_MRP']/train_df['Item_Weight']
test_df['Item_MRP_Per_Unit_Weight'] = test_df['Item_MRP']/test_df['Item_Weight']
    
    
# Select features and target
categorical_cols = [ 'Outlet_Combined']
numerical_cols = ['Item_MRP', 'Outlet_Total_Sales', 'Outlet_Age']

# Ensure categorical columns are treated as category dtype
for col in categorical_cols:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

# Select X and y
X = train_df[numerical_cols + categorical_cols]
y = train_df['Item_Outlet_Sales']
X_test = test_df[numerical_cols + categorical_cols]

# Split data
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, random_state=42)


def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.05),  # Wider range
        'num_leaves': trial.suggest_int('num_leaves', 50, 200),  # More flexibility
        'max_depth': trial.suggest_int('max_depth', 3, 10),  # Slightly deeper trees
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'reg_lambda': trial.suggest_float('reg_lambda', 5, 12),  # Stronger L2 regularization
        'reg_alpha': trial.suggest_float('reg_alpha', 5, 12),  # Stronger L1 regularization
        'n_estimators': 5000,  # Increased for better convergence
        'verbosity': -1,  # Reduce logs for speed
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 50),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 0.9),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 0.9),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10)
    }

    model = lgb.LGBMRegressor(**params)

    eval_result = {}
    model.fit(
        X_train, y_train, 
        eval_set=[(X_validate, y_validate)], 
        eval_metric='rmse',
        categorical_feature=categorical_cols,  # Pass categorical features!
        callbacks=[lgb.early_stopping(100, verbose=False), lgb.record_evaluation(eval_result)]
    )
    
    preds = model.predict(X_validate)
    rmse = mean_squared_error(y_validate, preds, squared=False)
    return rmse

# Run Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  # Increase trials for better tuning

print("Best Parameters:", study.best_params)


[I 2025-03-18 00:50:39,772] A new study created in memory with name: no-name-ace8932b-8b35-4e4d-b4b4-165a907829f7
[I 2025-03-18 00:50:39,918] Trial 0 finished with value: 1018.2005109846273 and parameters: {'learning_rate': 0.03536606799946188, 'num_leaves': 162, 'max_depth': 6, 'min_child_samples': 37, 'colsample_bytree': 0.6708243338349053, 'subsample': 0.7260553181160913, 'reg_lambda': 5.814468070119192, 'reg_alpha': 6.450629242654854, 'min_data_in_leaf': 27, 'feature_fraction': 0.8144948601808867, 'bagging_fraction': 0.4607273729286345, 'bagging_freq': 2}. Best is trial 0 with value: 1018.2005109846273.
[I 2025-03-18 00:50:41,993] Trial 1 finished with value: 1025.368636652261 and parameters: {'learning_rate': 0.0015206997475358374, 'num_leaves': 71, 'max_depth': 8, 'min_child_samples': 93, 'colsample_bytree': 0.8237730462138798, 'subsample': 0.8979623851663447, 'reg_lambda': 10.691190700867864, 'reg_alpha': 10.731866000851939, 'min_data_in_leaf': 14, 'feature_fraction': 0.78067255

[I 2025-03-18 00:50:45,871] Trial 17 finished with value: 1017.4832487046822 and parameters: {'learning_rate': 0.011567129368547693, 'num_leaves': 76, 'max_depth': 4, 'min_child_samples': 15, 'colsample_bytree': 0.6157857219235067, 'subsample': 0.8114681790445855, 'reg_lambda': 8.009074804015357, 'reg_alpha': 8.125912085396953, 'min_data_in_leaf': 38, 'feature_fraction': 0.5506004804559441, 'bagging_fraction': 0.5343842141099407, 'bagging_freq': 5}. Best is trial 8 with value: 1016.3542985858571.
[I 2025-03-18 00:50:47,787] Trial 18 finished with value: 1018.1338159372181 and parameters: {'learning_rate': 0.0011314132236276177, 'num_leaves': 93, 'max_depth': 5, 'min_child_samples': 70, 'colsample_bytree': 0.7650843975767153, 'subsample': 0.7869029347125733, 'reg_lambda': 6.395033449083058, 'reg_alpha': 6.971274893460405, 'min_data_in_leaf': 32, 'feature_fraction': 0.8428055285917246, 'bagging_fraction': 0.6181371804783076, 'bagging_freq': 8}. Best is trial 8 with value: 1016.3542985858

[I 2025-03-18 00:50:50,225] Trial 34 finished with value: 1014.8714035155638 and parameters: {'learning_rate': 0.04881458025112454, 'num_leaves': 141, 'max_depth': 4, 'min_child_samples': 46, 'colsample_bytree': 0.7867748677120606, 'subsample': 0.8944626078308132, 'reg_lambda': 10.493555843189272, 'reg_alpha': 10.272441874517941, 'min_data_in_leaf': 40, 'feature_fraction': 0.6568051713593874, 'bagging_fraction': 0.4022036412715186, 'bagging_freq': 2}. Best is trial 22 with value: 1014.158488506265.
[I 2025-03-18 00:50:50,369] Trial 35 finished with value: 1016.8013599340173 and parameters: {'learning_rate': 0.04403696256749852, 'num_leaves': 138, 'max_depth': 4, 'min_child_samples': 40, 'colsample_bytree': 0.7863070209097736, 'subsample': 0.8938182616189533, 'reg_lambda': 10.533456820087311, 'reg_alpha': 10.301074525113641, 'min_data_in_leaf': 40, 'feature_fraction': 0.5903126061063407, 'bagging_fraction': 0.450320700649765, 'bagging_freq': 2}. Best is trial 22 with value: 1014.1584885

Best Parameters: {'learning_rate': 0.047673695659454375, 'num_leaves': 142, 'max_depth': 3, 'min_child_samples': 51, 'colsample_bytree': 0.8388305149694122, 'subsample': 0.8470246164321973, 'reg_lambda': 9.33772086438144, 'reg_alpha': 11.948450889637312, 'min_data_in_leaf': 46, 'feature_fraction': 0.7309646028654864, 'bagging_fraction': 0.49858443214156756, 'bagging_freq': 3}


In [109]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Best parameters from Optuna
best_params = study.best_params.copy()  # Copy the original dictionary
best_params.update({  
    'objective': 'regression',  
    'metric': 'rmse',  
    'n_estimators': 2000  
})



# Train model
model = LGBMRegressor(**best_params)
model.fit(X_train, y_train, 
          eval_set=[(X_validate, y_validate)], 
          eval_metric='rmse', 
          callbacks=[lgb.early_stopping(50, verbose=False)])

# Predict on training & validation sets
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_validate)

# Evaluate performance
def evaluate(y_true, y_pred, dataset):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n✅ Model Performance on {dataset}:")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R² Score: {r2:.4f}")

# Show results
evaluate(y_train, y_train_pred, "Training Set")
evaluate(y_validate, y_valid_pred, "Validation Set")

# Predict on test set
y_test_preds = model.predict(X_test)

# Create submission file
submission = test_df[['Item_Identifier', 'Outlet_Identifier']].copy()
submission['Item_Outlet_Sales'] = y_test_preds

submission['Item_Outlet_Sales'] = y_test_preds.clip(min=0)

# Save CSV
submission.to_csv('../submission/submission_lgb.csv', index=False)

print("Download your submission file: /mnt/data/submission_lgb.csv")
#1150.2858411057 rank#790



✅ Model Performance on Training Set:
MAE: 763.8165
MSE: 1168085.2705
RMSE: 1080.7799
R² Score: 0.6051

✅ Model Performance on Validation Set:
MAE: 716.6091
MSE: 1028456.0442
RMSE: 1014.1282
R² Score: 0.6216
Download your submission file: /mnt/data/submission_xgboost.csv
